In [1]:
from __future__ import print_function
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

import cntk as C
import cntk.tests.test_utils

%matplotlib inline

In [2]:
input_dim = 64
num_output_classes = 10

In [4]:
def create_reader(path, is_training, input_dim, num_output_classes):
    
    label_stream = C.io.StreamDef(field='labels', shape=num_output_classes, is_sparse=False)
    feature_stream = C.io.StreamDef(field='features', shape=input_dim, is_sparse=False)
    
    desirelizer = C.io.CTFDeserializer(path, C.io.StreamDefs(labels=label_stream, features=feature_stream))
    
    return C.io.MinibatchSource(desirelizer,
                               randomize=is_training, max_sweeps=C.io.INFINITELY_REPEAT if is_training else 1)

In [6]:
data_found = False

for data_dir in [os.path.join("..", "Examples", "Image", "DataSets", "MNIST"),
                os.path.join("data", "MNIST")]:
    train_file = os.path.join(data_dir, "Train-28x28_cntk_text.txt")
    test_file = os.path.join(data_dir, "Test-28x28_cntk_text.txt")
    if os.path.isfile(train_file) and os.path.isfile(test_file):
        data_found = True
        break
        
if not data_found:
    raise ValueError("Please generate the data by completing CNTK 103 Part A")
    
print("Data directory is {0}".format(data_dir))

Data directory is data\MNIST


In [7]:
input = C.input_variable(input_dim)
label = C.input_variable(num_output_classes)

In [10]:
def create_model(features):
    with C.layers.default_options(init=C.glorot_uniform()):
        r = C.layers.Dense(num_output_classes, activation=None)(features)
        
    return r

In [11]:
z = create_model(input)